In [1]:
import os
import numpy as np

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchvision.transforms as T

import cv2
import matplotlib.pyplot as plt
from PIL import Image

from skimage.segmentation import slic
from fast_slic import Slic
from sklearn.cluster import KMeans

from mmseg.datasets.transforms import *
from mmseg.datasets import PascalVOCDataset
from mmengine.structures import PixelData


from mmengine.config import Config, DictAction
from mmengine.runner import Runner
import itertools
import time

from modelscope import (
    snapshot_download, AutoModelForCausalLM, AutoTokenizer, GenerationConfig
)

/opt/conda/envs/mm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-08 13:16:10,629 - modelscope - INFO - PyTorch version 1.13.1+cu117 Found.
2023-11-08 13:16:10,633 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-11-08 13:16:10,698 - modelscope - INFO - Loading done! Current index file version is 1.9.4, with md5 15e81bede39dd83de9c6d0777d4ba7ab and a total number of 945 components indexed


In [2]:
model_id = 'qwen/Qwen-VL-Chat'
revision = 'v1.0.0'
model_dir = '../Qwen-VL-Chat'
model = AutoModelForCausalLM.from_pretrained(model_dir, device_map="cuda", trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
if not hasattr(tokenizer, 'model_dir'):
    tokenizer.model_dir = model_dir

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Loading checkpoint shards: 100%|██████████| 10/10 [00:26<00:00,  2.65s/it]


In [33]:
classes=['background', 'aeroplane', 'bicycle', 'bird', 'boat',
        'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable',
        'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep',
        'sofa', 'train', 'tvmonitor']
classes_str = ""
for name in classes:
    classes_str += (name + ", ")
classes_str = classes_str[:-2]


prompt_template = "<img>{}</img>The green mask in the figure covers part of an object, \
please analyze what is the most likely category of this object? Please select from the categories given below: {}.\
Please distinguish as many categories of objects as possible, and do not be affected by the main objects in the figure."

name2id, id2name = dict(), dict()

for id, class_name in enumerate(classes):
    name2id[class_name] = id
    id2name[id] = class_name

In [4]:
# class FeatureHooker:
#     def __init__(self, layer):
#         self.layer = layer
#         self.fea = None
#         self.handle = None

#         self.register_hook()
    
#     def hook_fn(self, m, fea_in, fea_out):
#         self.fea = fea_out
        
#     def register_hook(self):
#         self.handle = self.layer.register_forward_hook(self.hook_fn)

# hooker = FeatureHooker(layer=model.transformer.visual.transformer.resblocks[47])

feature_l = [None]
def hook_fn(m, fea_in, fea_out):
    feature_l[0] = fea_out.detach().cpu()

handle = model.transformer.visual.transformer.resblocks[47].register_forward_hook(hook_fn)

In [5]:
queries = ['<img>{}</img>Describe this image.'.format(img_path)]
responses, history = model.chat(tokenizer=tokenizer, queries=queries, history=None)

In [62]:
img_path = '/remote-home/zhangjiacheng/RVP/data/rendered_img/2007_001175/5.jpg'
queries = [prompt_template.format(img_path, classes_str)]
responses, history = model.chat(tokenizer=tokenizer, queries=queries, history=None)
print (responses[0])

mask


In [20]:
# K-Means Cluter
device = 'cuda:0'
# feature = hooker.fea[:, 0].cpu().float()
feature = feature_l[0][:, 0].float()
num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters)
cluster_ids = kmeans.fit_predict(feature)

cluster_img = np.zeros((448, 448, 3))

for i in range(32):
    for j in range(32):
        patch_label = cluster_ids[i * 32 + j]
        # color = plt.cm.tab10(patch_label / 10)
        # cluster_img[i*14:(i+1)*14, j*14:(j+1)*14] = color[:3]
        cluster_img[i*14:(i+1)*14, j*14:(j+1)*14] = patch_label

# superpixels_masks.shape: (ids, C, H, W)
masks = []
for id in np.unique(cluster_ids):
    mask = (cluster_img == id)
    masks.append(torch.tensor(mask))
masks = torch.stack(masks).permute(0, 3, 1, 2)

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = torch.tensor(img.transpose(2, 0, 1))
img_repeated = T.Resize((448, 448))(img)[None].expand_as(masks)

/opt/conda/envs/mm/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [1]:
masks.shape
import torchvision.transforms.functional as TF
resized = TF.resize(masks, (500, 375))
resized[0, 0]
masks[0, 0]

t = resized[0, 0].numpy().astype(np.uint8)
t.shape
plt.imshow(t)
# plt.imshow(masks[0, 0].numpy().astype(np.uint8))

resized

NameError: name 'masks' is not defined

In [2]:
color = torch.tensor([[255, 0, 0], [0, 255, 0], [0, 0, 255]]) # R G B
Red = color[0].reshape(1, 3, 1, 1)
Green = color[1].reshape(1, 3, 1, 1)
Blue = color[2].reshape(1, 3, 1, 1)

mask_imgs = torch.zeros_like(img_repeated)
remain_imgs = torch.zeros_like(img_repeated)
mask_imgs[masks] = img_repeated[masks]
remain_imgs[masks.logical_not()] = img_repeated[masks.logical_not()]

# Render images
# Mix-up (Background brightness unchanged)
color_mode = 'G'
if color_mode == 'R':
    rendered_imgs = (mask_imgs * 0.6 + masks * Red * 0.4) + remain_imgs
    # rendered_imgs = img_repeated * 0.6 + masks * Red * 0.4
elif color_mode == 'G':
    rendered_imgs = (mask_imgs * 0.6 + masks * Green * 0.4) + remain_imgs
    # rendered_imgs = img_repeated * 0.6 + masks * Green * 0.4
elif color_mode == 'B':
    rendered_imgs = (mask_imgs * 0.6 + masks * Blue * 0.4) + remain_imgs
    # rendered_imgs = img_repeated * 0.6 + masks * Blue * 0.4
else:
    raise ValueError('Color not supported: {}'.format(color_mode))

NameError: name 'torch' is not defined

In [21]:
pred = cv2.imread('/remote-home/zhangjiacheng/RVP/data/sem_seg_preds/scikit30/G/2007_001175.jpg')
# pred = np.array(Image.open('/remote-home/zhangjiacheng/RVP/data/sem_seg_preds/scikit30/G/2007_001175.jpg'))
# print (pred.shape)
# print (pred[..., 0])
# for i in range(pred.shape[0]):
#     print (pred[i])
print (np.unique(pred))
pred1 = np.load('./test.npy')
print (np.unique(pred1))
pred1[None].shape

[ 0  1  2  3  4  5 12 13 14 15 16 17 18 19 20 21 22 23]
[ 0 18]


(1, 375, 500)

In [18]:

np.unique(pred2)

array([ 0,  1,  2,  3,  4,  5, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23], dtype=uint8)